In [ ]:
!pip install albumentations

# **Deep Learning Exploration**

## **Load Data**

In [10]:
root_dir = "/Users/macano/Desktop/Tesis_code/GeoAI"
base_dir = root_dir
print(base_dir)

/Users/macano/Desktop/Tesis_code/GeoAI


## **Data Augmentation**

En este apartado se genera la aumentación de datos, buscando que la aumentación sea aplicada a ambas clases se crean dos generadores con las mismas caracteristicas y posterior a eso se embeben en un iterable. Esta aumentación se le aplica a cada una de las imagenes y mascaras y se obtiene dos carpetas una con mascaras aumentadas y una con imagenes aumentadas, que posterior a esto se pueden usar para entrenar redes neuronales. 

In [11]:
import os
import shutil

# Carpeta de origen
src_folder = os.path.join(base_dir, 'Training')

# Carpeta de destino principal
dst_folder = os.path.join(base_dir, 'Train_v2')
if not os.path.exists(dst_folder):
    os.makedirs(dst_folder)

# Carpeta de destino para imágenes reales
dst_real_folder = os.path.join(dst_folder, "real_img")
if not os.path.exists(dst_real_folder):
    os.makedirs(dst_real_folder)

# Carpeta de destino para máscaras
dst_mask_folder = os.path.join(dst_folder, "mask")
if not os.path.exists(dst_mask_folder):
    os.makedirs(dst_mask_folder)

# Contadores para reenumerar
real_count = 1
mask_count = 1

# Iterar a través de las carpetas de origen
for i in range (1,5):
  tif_path = os.path.join(src_folder, "real_img", f'TIF_{i}')
  mask_path = os.path.join(src_folder, "mask", f'TIF_{i}')

    # Copiar imágenes reales a la carpeta de destino
  for img_file in os.listdir(tif_path):
    real_dst_path = os.path.join(dst_real_folder, f"{real_count}.png")
    mask_dst_path = os.path.join(dst_mask_folder, f"{mask_count}.png")
    shutil.copy2(
              os.path.join(tif_path, img_file),
              real_dst_path
              )
    shutil.copy2(
              os.path.join(mask_path, img_file),
              mask_dst_path
              )
    real_count += 1
    mask_count += 1

print("Listo!")

Listo!


In [37]:
import os
import cv2
from tqdm import tqdm
from glob import glob
from albumentations import CenterCrop, RandomRotate90, GridDistortion, HorizontalFlip, VerticalFlip, ShiftScaleRotate, ElasticTransform, RandomBrightnessContrast, HueSaturationValue, GaussianBlur, RandomGamma, ChannelShuffle, CLAHE, RandomShadow

output_dir = os.path.join(base_dir, 'augmented_images_v2')

def load_data():
     images = sorted([os.path.join(base_dir, 'Train_v2/real_img', i) for i in os.listdir(os.path.join(base_dir, "Train_v2/real_img"))])     
     masks = sorted([os.path.join(base_dir, 'Train_v2/mask', i) for i in os.listdir(os.path.join(base_dir, "Train_v2/mask"))])
     return images, masks

In [30]:
def augment_data_geometric(images, masks, save_path, augment=True):
    H = 256
    W = 256
    
    for x, y in tqdm(zip(images, masks), total=len(images)):
        name = x.split("/")[-1].split(".")
        """ Extracting the name and extension of the image and the mask. """
        image_name = name[0]
        image_extn = name[1]

        name = y.split("/")[-1].split(".")
        mask_name = name[0]
        mask_extn = name[1]

        """ Reading image and mask. """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)

        """ Augmentation """
        if augment == True:
            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented['image']
            y1 = augmented['mask']

            aug = GridDistortion(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented['image']
            y2 = augmented['mask']

            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x3 = augmented['image']
            y3 = augmented['mask']

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x4 = augmented['image']
            y4 = augmented['mask']

            aug = ShiftScaleRotate(p=1.0, shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=cv2.BORDER_CONSTANT)
            augmented = aug(image=x, mask=y)
            x5 = augmented['image']
            y5 = augmented['mask']

            aug = ElasticTransform(p=1.0, alpha=1, sigma=50, alpha_affine=50)
            augmented = aug(image=x, mask=y)
            x6 = augmented['image']
            y6 = augmented['mask']

            save_images = [x, x1, x2, x3, x4, x5, x6]
            save_masks =  [y, y1, y2, y3, y4, y5, y6]

        else:
            save_images = [x]
            save_masks = [y]

        """ Saving the image and mask. """
        idx = 0
        for i, m in zip(save_images, save_masks):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            if len(images) == 1:
                tmp_img_name = f"{image_name}.{image_extn}"
                tmp_mask_name = f"{mask_name}.{mask_extn}"

            else:
                tmp_img_name = f"{image_name}_{idx}.{image_extn}"
                tmp_mask_name = f"{mask_name}_{idx}.{mask_extn}"

            image_path = os.path.join(save_path, "images", tmp_img_name)
            mask_path = os.path.join(save_path, "masks", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            idx += 1

In [49]:
def augment_data_espectral(images, masks, save_path, augment=True):
    H = 256
    W = 256
    
    for x, y in tqdm(zip(images, masks), total=len(images)):
        name = x.split("/")[-1].split(".")
        """ Extracting the name and extension of the image and the mask. """
        image_name = name[0]
        image_extn = name[1]

        name = y.split("/")[-1].split(".")
        mask_name = name[0]
        mask_extn = name[1]

        """ Reading image and mask. """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)

        """ Augmentation """
        if augment == True:
            #cambios aleatorios en el brillo y el contraste de la imagen.
            aug = RandomBrightnessContrast(p=1.0, brightness_limit=0.5, contrast_limit=0.5)
            augmented = aug(image=x, mask=y)
            x1 = augmented['image']
            y1 = y

            #cambios aleatorios en la tonalidad, la saturación y el brillo de la imagen.
            aug = HueSaturationValue(p=1.0, hue_shift_limit=50, sat_shift_limit=50, val_shift_limit=50)
            augmented = aug(image=x, mask=y)
            x2 = augmented['image']
            y2 = y
            
            # filtro Gaussiano aleatorio a la imagen.
            aug = GaussianBlur(p=1.0, blur_limit=(3, 7))
            augmented = aug(image=x, mask=y)
            x3 = augmented['image']
            y3 = y

            # cambio aleatorio en la intensidad de la imagen utilizando la función de corrección gamma.
            aug = RandomGamma(p=1.0, gamma_limit=(80, 120))
            augmented = aug(image=x, mask=y)
            x4 = augmented['image']
            y4 = y

            # método de mejora de contraste de imagen llamado Adaptive Histogram Equalization 
            # (AHE) que normaliza la distribución del histograma de la imagen.
            aug = CLAHE(p=1.0, clip_limit=2.0, tile_grid_size=(8, 8))
            augmented = aug(image=x, mask=y)
            x5 = augmented['image']
            y5 = y
                        

            save_images = [x, x1, x2, x3, x4, x5]
            save_masks =  [y, y1, y2, y3, y4, y5]

        else:
            save_images = [x]
            save_masks = [y]

        """ Saving the image and mask. """
        idx = 0
        for i, m in zip(save_images, save_masks):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            if len(images) == 1:
                tmp_img_name = f"{image_name}.{image_extn}"
                tmp_mask_name = f"{mask_name}.{mask_extn}"

            else:
                tmp_img_name = f"{image_name}_{idx}.{image_extn}"
                tmp_mask_name = f"{mask_name}_{idx}.{mask_extn}"

            image_path = os.path.join(save_path, "images", tmp_img_name)
            mask_path = os.path.join(save_path, "masks", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            idx += 1

## **Geometric Augmentation**

In [32]:
images, masks = load_data()
save_path = os.path.join(base_dir, 'aum_geometric')
augment_data_geometric(images, masks, save_path, augment=True)

100%|██████████| 532/532 [01:59<00:00,  4.44it/s]


## **Espectral Augmentation**

In [50]:
images, masks = load_data()
save_path = os.path.join(base_dir, 'aum_espectral')
augment_data_espectral(images, masks, save_path, augment=True)

100%|██████████| 532/532 [00:21<00:00, 25.11it/s]


In [48]:
len(os.listdir(os.path.join(base_dir, 'aum_espectral/masks')))

3724